In [106]:
import pandas as pd
import numpy as np

In [107]:
file_path = r'C:\Users\PC\Desktop\Projects\User Journey Analysis in Python Project\project-files-user-journey-analysis-in-python (1)\user_journey_raw.csv';
df= pd.read_csv(file_path);
df

,user_id,session_id,subscription_type,user_journey
0,1516,2980231,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
1,1516,2980248,Annual,Other-Sign up-Sign up-Sign up-Sign up-Sign up-...
2,1516,2992252,Annual,Log in-Log in-Log in-Log in-Log in-Log in
3,1516,3070491,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
4,1516,3709807,Annual,Log in-Log in-Log in-Log in-Log in-Log in-Log ...
...,...,...,...,...
9930,509095,4487613,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...
9931,509095,4842565,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...
9932,509095,4843103,Annual,Other-Other
9933,509095,4845316,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...


In [108]:
df.dtypes

user_id               int64
session_id            int64
subscription_type    object
user_journey         object
dtype: object

## Data Preprocessing

In [109]:
# Make a copy of the DataFrame to avoid modifying the original
# Split the journey string into individual pages
# Initialize a new list to store the cleaned pages
# Iterate over pages, adding each page only if it's different from the last added
# Join the cleaned pages back into a single string
# Apply the function to each row in the user_journey column
# Display the cleaned DataFrame

def remove_page_duplicates(data ,target_column='user_journey'):
    df_copy = data.copy()
    def remove_sequential_duplicates(journey):
        if isinstance(journey , str):
            pages = journey.split('-')
            cleaned_pages = []
            for page in pages:
                if not  cleaned_pages or page!=cleaned_pages[-1] :
                    cleaned_pages.append(page)
            return '-'.join(cleaned_pages)
        else :
            return journey
    df_copy[target_column] = df_copy[target_column].apply(remove_sequential_duplicates)
    return df_copy



In [110]:
cleaned_df = remove_page_duplicates(df, 'user_journey')
cleaned_df

,user_id,session_id,subscription_type,user_journey
0,1516,2980231,Annual,Homepage-Log in-Other
1,1516,2980248,Annual,Other-Sign up-Log in
2,1516,2992252,Annual,Log in
3,1516,3070491,Annual,Homepage-Log in
4,1516,3709807,Annual,Log in
...,...,...,...,...
9930,509095,4487613,Annual,Other
9931,509095,4842565,Annual,Other
9932,509095,4843103,Annual,Other
9933,509095,4845316,Annual,Other


In [118]:
# Create a copy of the DataFrame to avoid modifying the original
# Sort the DataFrame by the group_column and session_id to ensure proper ordering within each user's sessions
# Define a function to concatenate journeys based on specified sessions and count_from
# If sessions is 'All', concatenate all journeys for the user
# If sessions is an integer, select journeys based on count_from
# Concatenate the selected journeys into one big string
# Group by the group_column and apply the concatenate_journeys function
# Rename the columns for clarity

def group_by(data , group_column='user_id' , target_column='user_journey' , sessions='All' , count_from='last'):
    df_copy = data.copy()
    df_copy = df_copy.sort_values(by=[group_column , 'session_id']).reset_index(drop=True)
    def concatenate_journeys(group):
        if sessions == 'All':
            selected_journey = group[target_column]
        else :
            num_sessions = int(sessions)
            if count_from == 'first':
                selected_journey = group[target_column].head(num_sessions)
            elif count_from == 'last':
                selected_journey = group [target_column].tail(num_sessions)
            else:
                raise ValueError ("Count_from must be either 'first' or 'last'") 
        return '-'.join(selected_journey)
    grouped_df = df_copy.groupby(group_column,group_keys=False).apply(concatenate_journeys).reset_index()
    grouped_df.columns = [group_column , target_column]

    return grouped_df

In [120]:
# Assuming your DataFrame is called df
# Example 1: Group all sessions for each user
# Example 2: Group only the first 10 sessions for each user
# Example 3: Group only the last 3 sessions for each user
# Display the result


group_df_all = group_by(df , 'user_id' , 'user_journey' , sessions='All')
#print(group_df_all.head())

group_df_10 = group_by(df , 'user_id' , 'user_journey' , sessions=10 , count_from='last')
group_df_10

grouped_df_last_3 = group_by(df, group_column='user_id', target_column='user_journey', sessions=3, count_from='last')
grouped_df_last_3


C:\Users\PC\AppData\Local\Temp\ipykernel_17856\445174935.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = df_copy.groupby(group_column,group_keys=False).apply(concatenate_journeys).reset_index()
C:\Users\PC\AppData\Local\Temp\ipykernel_17856\445174935.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = df_copy.groupby(group_column,group_keys=False).apply(concatenate_journeys).

,user_id,user_journey
0,1516,Checkout-Checkout-Checkout-Checkout-Checkout-C...
1,3395,Homepage-Pricing-Pricing-Checkout-Checkout-Che...
2,10107,Checkout-Checkout-Checkout-Checkout-Checkout-C...
3,11145,Homepage-Log in-Log in-Log in-Homepage-Log in-...
4,12400,Other-Career track certificate-Career track ce...
...,...,...
1345,509060,Other-Other-Other-Other-Other-Other-Other-Othe...
1346,509061,Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coup...
1347,509085,Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coup...
1348,509095,Other-Other-Other-Other-Other-Other-Other-Othe...


In [121]:
grouped_df_last_3


,user_id,user_journey
0,1516,Checkout-Checkout-Checkout-Checkout-Checkout-C...
1,3395,Homepage-Pricing-Pricing-Checkout-Checkout-Che...
2,10107,Checkout-Checkout-Checkout-Checkout-Checkout-C...
3,11145,Homepage-Log in-Log in-Log in-Homepage-Log in-...
4,12400,Other-Career track certificate-Career track ce...
...,...,...
1345,509060,Other-Other-Other-Other-Other-Other-Other-Othe...
1346,509061,Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coup...
1347,509085,Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coup...
1348,509095,Other-Other-Other-Other-Other-Other-Other-Othe...
